In [11]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Get current working directory instead of __file__
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from CoAtNetModel import CoAtNetWheatModel
from dataLoaderFunc import loadSplitData, createLoader

In [12]:
# ✅ Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for batch_idx, (rgb_batch, dsm_batch, label_batch) in enumerate(train_loader):
            rgb_batch, dsm_batch, label_batch = rgb_batch.to(device), dsm_batch.to(device), label_batch.to(device)

            optimizer.zero_grad()
            outputs = model(rgb_batch, dsm_batch)
            loss = criterion(outputs, label_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            if batch_idx % 50 == 0:
                print(f"Epoch {epoch+1}/{num_epochs} | Batch {batch_idx}/{len(train_loader)} | Loss: {loss.item():.4f}")

        train_loss /= len(train_loader)

        # ✅ Compute Validation Loss
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for rgb_batch, dsm_batch, label_batch in val_loader:
                rgb_batch, dsm_batch, label_batch = rgb_batch.to(device), dsm_batch.to(device), label_batch.to(device)
                outputs = model(rgb_batch, dsm_batch)
                loss = criterion(outputs, label_batch)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        scheduler.step(val_loss)

        print(f"✅ Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # ✅ Save Best Model
        torch.save(model.state_dict(), "coatnet_wheat_model.pth")


# ✅ Test the model on validation set
def test_model(model, test_loader):
    model.eval()
    predictions, actuals = [], []

    with torch.no_grad():
        for rgb_batch, dsm_batch, label_batch in test_loader:
            rgb_batch, dsm_batch = rgb_batch.to(device), dsm_batch.to(device)
            outputs = model(rgb_batch, dsm_batch)
            predictions.extend(outputs.cpu().numpy().flatten())
            actuals.extend(label_batch.cpu().numpy().flatten())

    return predictions, actuals


In [13]:
train_df, val_df, test_df = loadSplitData("RGB_DSM_totEarNum.csv")
train_loader, val_loader, test_loader = createLoader(train_df, val_df, test_df)

Train Size: 47840, Validation Size: 5980, Test Size: 5980
Train Batches: 2990, Validation Batches: 374, Test Batches: 374


In [14]:
# ✅ Initialize Model, Loss Function, and Optimizer

# ✅ Universal device selection (works on both Mac M2 & Windows with RTX 4060)
if torch.backends.mps.is_available():
    device = "mps"  # ✅ Use Apple Metal (Mac M1/M2)
    torch.set_default_tensor_type(torch.FloatTensor)
elif torch.cuda.is_available():
    device = "cuda"  # ✅ Use NVIDIA CUDA (Windows RTX 4060)
else:
    device = "cpu"  # ✅ Default to CPU if no GPU is available
print(f"✅ Using device: {device}")

model = CoAtNetWheatModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)


✅ Using device: cuda


In [10]:
# ✅ Start Training
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10)

Epoch 1/10 | Batch 0/2990 | Loss: 120645.7656
Epoch 1/10 | Batch 50/2990 | Loss: 10883.1357
Epoch 1/10 | Batch 100/2990 | Loss: 11027.0078
Epoch 1/10 | Batch 150/2990 | Loss: 2443.8435
Epoch 1/10 | Batch 200/2990 | Loss: 9839.1211
Epoch 1/10 | Batch 250/2990 | Loss: 2958.2566
Epoch 1/10 | Batch 300/2990 | Loss: 4616.9434
Epoch 1/10 | Batch 350/2990 | Loss: 1950.3430
Epoch 1/10 | Batch 400/2990 | Loss: 2197.8569
Epoch 1/10 | Batch 450/2990 | Loss: 3891.9268
Epoch 1/10 | Batch 500/2990 | Loss: 3604.5291
Epoch 1/10 | Batch 550/2990 | Loss: 3274.6655
Epoch 1/10 | Batch 600/2990 | Loss: 4195.3418
Epoch 1/10 | Batch 650/2990 | Loss: 4345.6328
Epoch 1/10 | Batch 700/2990 | Loss: 1331.4562
Epoch 1/10 | Batch 750/2990 | Loss: 1780.7058
Epoch 1/10 | Batch 800/2990 | Loss: 2858.3533
Epoch 1/10 | Batch 850/2990 | Loss: 2642.7563
Epoch 1/10 | Batch 900/2990 | Loss: 1735.2305
Epoch 1/10 | Batch 950/2990 | Loss: 3640.8032
Epoch 1/10 | Batch 1000/2990 | Loss: 3499.1736
Epoch 1/10 | Batch 1050/2990 | L

In [15]:
# ✅ Load the saved model
model.load_state_dict(torch.load("coatnet_wheat_model.pth"))
model.eval()

# Run test
preds, actuals = test_model(model, test_loader)

# ✅ Print sample predictions
for p, a in zip(preds[:10], actuals[:10]):
    print(f"Predicted: {p:.2f}, Actual: {a:.2f}")

Predicted: 378.04, Actual: 368.00
Predicted: 563.03, Actual: 552.00
Predicted: 458.03, Actual: 454.00
Predicted: 298.36, Actual: 281.00
Predicted: 474.43, Actual: 472.00
Predicted: 433.04, Actual: 425.00
Predicted: 303.64, Actual: 295.00
Predicted: 457.78, Actual: 430.00
Predicted: 410.50, Actual: 392.00
Predicted: 133.23, Actual: 129.00
